In [1]:
import sys
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms

from torch.utils.data import Dataset, DataLoader
from PIL import Image

%reload_ext autoreload
%autoreload 2

sys.path.append(r"D:\Code\ML\face-recognition-system")


In [2]:
from src.data.data_loader import DigiFace
from src.models.partfVit import PartFVitWithLandmark
from src.losses.cosface import CosFaceLoss

In [3]:
pre_process = transforms.Compose(
    [
        transforms.Resize((112, 112)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
    ]
)

train_data = DigiFace(
    path=r"D:\Code\ML\face-recognition-system\data\train", transform=pre_process
)

train_loader = DataLoader(train_data, batch_size=1, shuffle=True)

In [4]:
class SimpleLinearLayer(nn.Module):
    def __init__(self, in_features, out_features):
        super(SimpleLinearLayer, self).__init__()
        self.linear = nn.Linear(in_features, out_features)
        self.relu = nn.ReLU()
        self.fc = nn.Linear(out_features, out_features)

        # Initialize the linear layers with random initialization
        nn.init.xavier_uniform_(self.linear.weight)
        nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        return self.fc(self.relu(self.linear(x)))


model = PartFVitWithLandmark(
    num_identites=10,
    num_landmarks=49,
    patch_size=28,
    in_channels=3,
    image_size=112,
    feat_dim=768,
    mlp_dim=2048,
    num_heads=12,
    num_layers=12,
    dropout=0.4,
).to("cuda:0")
classifier = SimpleLinearLayer(in_features=768, out_features=10).to("cuda:0")
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
classifier_optimizer = torch.optim.Adam(classifier.parameters(), lr=0.001)

c:\Users\Kshitij Sharma\.conda\envs\ml\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Kshitij Sharma\.conda\envs\ml\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
for epoch in range(50):
    for i, (image, label) in enumerate(train_loader):
        image = image.to("cuda:0")
        label = label.to("cuda:0")

        optimizer.zero_grad()
        classifier_optimizer.zero_grad()

        feature = model(image)
        output = classifier(feature)

        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        classifier_optimizer.step()

        print(torch.argmax(output[0][:10], dim=0), label[0])

        print(f"prediction: {output.argmax(dim=1).item()} label: {label.item()}")
        # print(f"Epoch: {epoch} Iteration: {i} Loss: {loss.item()}")

    # break

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
